In [27]:
# Required Libraries
from fpdf import FPDF
import os
import pandas as pd

# --- File Paths ---
png_logo_path = "Image_Scrape/viva_energy_logo_converted.png"  # Converted PNG path

# --- Load Commentary from .txt file ---
commentary_text = ""
with open("Commentary.txt", "r", encoding="utf-8") as file:
    commentary_text = file.read()

# --- Clean the commentary text to fix Unicode issues ---
commentary_text = commentary_text.replace('‘', "'").replace('’', "'")
commentary_text = commentary_text.replace('“', '"').replace('”', '"')

# --- Parse basic details ---
lines = commentary_text.split('\n')
company_name = lines[1].split(":")[1].strip()
ticker = lines[2].split(":")[1].strip()
sector = lines[3].split(":")[1].strip()
date = lines[4].split(":")[1].strip()

# --- Prepare sections ---
content = '\n'.join(lines[5:])
sections = {}
current_section = None
for line in content.split('\n'):
    if line.strip() in ["Executive Summary", "Company Overview", "Valuation", "Investment View"]:
        current_section = line.strip()
        sections[current_section] = ""
    elif current_section:
        sections[current_section] += line.strip() + " "

# --- Create PDF ---
pdf = FPDF(orientation='P', unit='mm', format='A4')
pdf.add_page()

# --- Add White Margin then Header Rectangle ---
pdf.set_fill_color(255, 255, 255)  # White background first
pdf.rect(0, 0, 210, 40, 'F')

pdf.set_fill_color(90, 90, 90)  # Mid grey
pdf.rect(10, 5, 190, 30, 'F')  # 10mm left/right margins

# --- Insert Viva Energy Logo (Shifted left slightly) ---
if os.path.exists(png_logo_path):
    pdf.image(png_logo_path, x=165, y=9, w=28)  # Logo shifted left and resized smaller
else:
    print("Warning: Logo PNG not found!")

# --- Header Text Setup (shifted to center and tighter) ---
pdf.set_text_color(255, 255, 255)
pdf.set_font('Arial', '', 9)
pdf.set_xy(15, 10)
pdf.cell(0, 4, f"{date}", ln=True)

pdf.set_font('Arial', 'B', 13)
pdf.set_x(15)
pdf.cell(0, 8, f"{company_name} ({ticker})", ln=True)

pdf.set_font('Arial', 'I', 9)
pdf.set_x(15)
pdf.cell(0, 5, "'Emerging stronger amid energy sector volatility'", ln=True)

# --- Sector + Exchange Info (Dropped lower below logo) ---
pdf.set_font('Arial', '', 7)
pdf.set_xy(155, 26)
pdf.cell(0, 3, f"GICS Sector: {sector}", ln=True)
pdf.set_x(155)
pdf.cell(0, 3, "Australian Securities Exchange (ASX)", ln=True)

# --- Recommendation Section ---
pdf.ln(4)
pdf.set_text_color(0, 0, 0)
pdf.set_font('Arial', '', 10)
pdf.set_x(10)
pdf.cell(40, 8, 'RECOMMENDATION:', ln=False)

pdf.set_text_color(255, 165, 0)  # Orange for HOLD
pdf.set_font('Arial', 'B', 10)
pdf.cell(0, 8, 'HOLD', ln=True)

pdf.ln(1)

# Insert Financial Summary Table Image
financial_summary_table_path = 'financial_summary_table.png'

if os.path.exists(financial_summary_table_path):
    # Force Y-position to be aligned nicely under RECOMMENDATION
    pdf.set_y(44)  # Adjust manually, you can tweak (e.g., 48, 50, etc.)
    pdf.set_x(10)  # Start at left margin
    pdf.image(financial_summary_table_path, x=pdf.get_x(), y=pdf.get_y(), w=95)  # Insert left side
    
    # Move cursor below the table
    table_height = 40  # Approximate height of table in mm (tweak if needed)
    pdf.set_xy(10, pdf.get_y() + table_height + 1)  # Move just below the table

    # Add footnote manually
    pdf.set_font('Arial', 'I', 6)
    pdf.set_text_color(100, 100, 100)  # Light grey footnote
    pdf.cell(90, 4, "All figures in A$ millions unless stated otherwise.", ln=True)

else:
    print("Warning: Financial Summary Table PNG not found!")

# Insert Ratios Summary Table BELOW the first one naturally (NO resetting Y)
ratios_summary_table_path = 'ratios_summary_table.png'

if os.path.exists(ratios_summary_table_path):
    pdf.ln(10)
    pdf.set_x(10)
    pdf.image(ratios_summary_table_path, x=10, y=pdf.get_y(), w=97)

    # Move cursor below ratios table
    ratios_table_height = 48.5  # Slightly less because it's slimmer
    pdf.set_xy(10, pdf.get_y() + ratios_table_height + 2)

    # Add footnote manually
    pdf.set_font('Arial', 'I', 6)
    pdf.set_text_color(100, 100, 100)  # Light grey footnote
    pdf.cell(90, 4, "Source: Yahoo Finance", ln=True)

else:
    print("Warning: Ratios Summary Table PNG not found!")

import openpyxl

# Load the newly uploaded Excel file
excel_path = "VEA_DCF.xlsx"
workbook = openpyxl.load_workbook(excel_path, data_only=True)
sheet = workbook.worksheets[1]  # Second sheet

# Extract required line items and their corresponding values
required_items = [
    "Unlevered FCF in last forecast period",
    "Unlevered FCF in next forecast period",
    "Terminal value",
    "Present value of terminal value",
    "Present value of stage 1",
    "Enterprise Value",
    "Cash and cash equivalents",
    "Borrowings",
    "Net debt",
    "Equity value",
    "Shares outstanding",
    "Share price",
    "Upside:"
]

# Extract matching data from the Excel file
extracted_data = []
for row in sheet.iter_rows(min_row=1, values_only=True):
    label = str(row[1]).strip() if row[1] else None
    value = row[6] if len(row) > 6 else None
    if label in required_items and value not in [None, ""]:
        # Format numbers nicely for presentation
        if isinstance(value, float):
            if "Upside" in label:
                formatted_value = f"{value:.2%}"
            elif value >= 1000:
                formatted_value = f"{value:,.0f}"
            else:
                formatted_value = f"{value:,.2f}"
        else:
            formatted_value = value
        extracted_data.append((label, formatted_value))


# Convert to DataFrame for visual confirmation
df_filtered = pd.DataFrame(extracted_data, columns=["Line Item", "Value"])

import matplotlib.pyplot as plt

def save_valuation_table_as_image(df, output_path='valuation_summary_table.png'):
    fig, ax = plt.subplots(figsize=(6, 5))  # Size adjusted based on content
    ax.axis('off')

    # Colors
    header_color = '#003366'
    row_colors = ['#e6f2ff', 'white']
    text_color = 'black'

    # Create the table anchored at top-left
    table = ax.table(
        cellText=df.values,
        colLabels=df.columns,
        rowLabels=None,
        loc='upper left',
        cellLoc='center',
    )

    # Styling
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.auto_set_column_width(col=list(range(len(df.columns))))
    table.scale(1.2, 1.2)

    for (i, j), cell in table.get_celld().items():
        if i == 0:
            cell.set_facecolor(header_color)
            cell.set_text_props(weight='bold', color='white', ha='center', va='center')
        else:
            cell.set_facecolor(row_colors[i % 2])
            if j == 0:  # Line Item column
                cell.set_text_props(color=text_color, ha='left', va='center')
            else:
                cell.set_text_props(color=text_color, ha='center', va='center')
        cell.set_linewidth(0.5)

    # Save as image
    plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0.05, transparent=True)
    plt.close()
    return output_path

# Save valuation table using provided styling
final_styled_path = save_valuation_table_as_image(df_filtered)
final_styled_path

# Insert Valuation Summary Table to the left of the VALUATION section
valuation_table_path = 'valuation_summary_table.png'

if os.path.exists(valuation_table_path):
    pdf.ln(8)
    x_pos = 9
    y_pos = pdf.get_y()

    # Insert image at current position
    pdf.image(valuation_table_path, x=x_pos, y=y_pos, w=73)

    # Estimate height of image manually (adjust if needed)
    image_height = 47  # millimeters; tweak this based on actual output
    pdf.set_y(y_pos + image_height + 1)  # Move below the image

    # Footnote
    pdf.set_font('Arial', 'I', 6)
    pdf.set_text_color(100, 100, 100)
    pdf.set_x(x_pos)
    pdf.cell(90, 4, "Source: Internal Valuation Model (VEA_DCF.xlsx)", ln=True)

    # Move cursor further down to clear space before commentary
    pdf.set_y(pdf.get_y() + 2)
else:
    print("Warning: Valuation Summary Table PNG not found!")

# --- Now move to Executive Summary (commentary) ---
# Set to right-hand side
pdf.set_xy(110, 38)

# --- Insert Commentary Sections ---
for section_title, section_body in sections.items():
    # Section Heading
    pdf.set_text_color(0, 51, 102)  # Dark blue
    pdf.set_font('Arial', 'B', 11)
    pdf.set_x(75)
    pdf.cell(0, 6, section_title.upper(), ln=True)

    # Draw underline
    pdf.set_draw_color(0, 51, 102)  # Blue line under heading
    pdf.set_line_width(0.5)
    pdf.line(75, pdf.get_y(), 199, pdf.get_y())  # From x=60 to right margin x=200
    pdf.ln(2)

    # Body Text
    pdf.set_text_color(0, 0, 0)
    pdf.set_font('Arial', '', 9)  # Slightly smaller body text
    pdf.set_x(75)
    pdf.multi_cell(125, 5, section_body.strip())  # Wider 140mm columns
    pdf.ln(2)

# --- Add New Page for VEA Price Chart ---
price_chart_path = 'vea_price_chart.png'

if os.path.exists(price_chart_path):
    pdf.add_page()  # Start a new page

    # Optional title or label
    pdf.set_text_color(0, 0, 0)
    pdf.set_font('Arial', 'B', 11)
    pdf.set_xy(10, 10)

    # Insert image
    pdf.image(price_chart_path, x=10, y=10, w=190)  # Adjust y=25 to leave space for title
else:
    print("Warning: Price chart image not found!")

# --- Output Final PDF ---
pdf.output('viva_energy_equity_report.pdf')
print("PDF Report created successfully!")


PDF Report created successfully!
